In [2]:
import numpy as np
import pandas as pd
import feather
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
df_train = pd.read_csv('../data/train.csv')

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_train.shape

(125497040, 6)

In [4]:
df_train.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25,103665,7.0,NaN
1,1,2013-01-01,25,105574,1.0,NaN
2,2,2013-01-01,25,105575,2.0,NaN
3,3,2013-01-01,25,108079,1.0,NaN
4,4,2013-01-01,25,108701,1.0,NaN


In [5]:
len(np.unique(df_train.store_nbr))

54

In [6]:
len(np.unique(df_train.item_nbr))

4036

In [7]:
np.min(df_train.date)

'2013-01-01'

In [8]:
np.max(df_train.date)

'2017-08-15'

In [9]:
df_items = pd.read_csv('../data/items.csv')

In [10]:
df_items.shape

(4100, 4)

In [11]:
df_items.head()

,item_nbr,family,class,perishable
0,96995,GROCERY I,1093,0
1,99197,GROCERY I,1067,0
2,103501,CLEANING,3008,0
3,103520,GROCERY I,1028,0
4,103665,BREAD/BAKERY,2712,1


In [12]:
len(np.unique(df_items.item_nbr))

4100

In [13]:
pd.value_counts(df_items.family)

GROCERY I                     1334
BEVERAGES                      613
CLEANING                       446
PRODUCE                        306
DAIRY                          242
PERSONAL CARE                  153
BREAD/BAKERY                   134
HOME CARE                      108
DELI                            91
MEATS                           84
HOME AND KITCHEN I              77
LIQUOR,WINE,BEER                73
FROZEN FOODS                    55
POULTRY                         54
HOME AND KITCHEN II             45
EGGS                            41
CELEBRATION                     31
PREPARED FOODS                  26
LAWN AND GARDEN                 26
LADIESWEAR                      21
LINGERIE                        20
AUTOMOTIVE                      20
BEAUTY                          19
PLAYERS AND ELECTRONICS         17
SCHOOL AND OFFICE SUPPLIES      15
GROCERY II                      14
PET SUPPLIES                    14
SEAFOOD                          8
MAGAZINES           

In [14]:
len(np.unique(df_items['class']))

337

In [15]:
pd.value_counts(df_items.perishable)

0    3114
1     986
Name: perishable, dtype: int64

In [16]:
len(set(df_train.item_nbr) - set(df_items.item_nbr))

0

In [17]:
len(set(df_items.item_nbr) - set(df_train.item_nbr))

64

In [18]:
list(set(df_items.item_nbr) - set(df_train.item_nbr))

[313094,
 2129678,
 2109969,
 2117783,
 2120216,
 2112280,
 766493,
 2015646,
 1464607,
 2128799,
 2124705,
 2127651,
 2130474,
 2130219,
 2134058,
 2112303,
 2130352,
 2127024,
 2127921,
 2129334,
 2127544,
 2120506,
 2120768,
 2131010,
 2132163,
 2129350,
 2127175,
 2129994,
 2112461,
 2112462,
 1467088,
 2132945,
 2127825,
 2130131,
 2129616,
 1467093,
 2130521,
 2130265,
 2132957,
 2132318,
 2130526,
 1921118,
 2127069,
 2112607,
 2129786,
 2134244,
 2129892,
 2127206,
 2121319,
 2126696,
 2120681,
 2129387,
 2129515,
 2131699,
 2131572,
 2127733,
 2116470,
 2125814,
 2127992,
 2130553,
 2120180,
 2128628,
 2118397,
 2129790]

In [19]:
not_in_train_item_ids = list(set(df_items.item_nbr) - set(df_train.item_nbr))

In [20]:
df_items1 = df_items[~df_items.item_nbr.isin(not_in_train_item_ids)]

In [21]:
df_items1.shape

(4036, 4)

In [22]:
df_train.isnull().sum()

id                    0
date                  0
store_nbr             0
item_nbr              0
unit_sales            0
onpromotion    21657651
dtype: int64

In [23]:
pd.value_counts(df_train.onpromotion)

False    96028767
True      7810622
Name: onpromotion, dtype: int64

In [24]:
df_items.isnull().sum()

item_nbr      0
family        0
class         0
perishable    0
dtype: int64

In [25]:
df_train.onpromotion.fillna('Unk', inplace=True)

In [26]:
pd.value_counts(df_train.onpromotion)

False    96028767
Unk      21657651
True      7810622
Name: onpromotion, dtype: int64

In [27]:
df_train1 = pd.merge(df_train, df_items1, how='left', on='item_nbr')

In [28]:
df_train1.shape

(125497040, 9)

In [29]:
df_train1.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
dtype: int64

In [30]:
df_train1.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1


In [31]:
df_stores = pd.read_csv('../data/stores.csv')

In [32]:
df_stores.shape

(54, 5)

In [33]:
df_stores.head()

,store_nbr,city,state,store_type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [34]:
len(set(df_train1.store_nbr))

54

In [35]:
pd.value_counts(df_stores.city)

Quito            18
Guayaquil         8
Santo Domingo     3
Cuenca            3
Latacunga         2
Manta             2
Machala           2
Ambato            2
Esmeraldas        1
Daule             1
Libertad          1
Guaranda          1
Ibarra            1
Loja              1
Babahoyo          1
Quevedo           1
Salinas           1
Puyo              1
El Carmen         1
Cayambe           1
Playas            1
Riobamba          1
Name: city, dtype: int64

In [36]:
pd.value_counts(df_stores.state)

Pichincha                         19
Guayas                            11
Santo Domingo de los Tsachilas     3
Manabi                             3
Azuay                              3
El Oro                             2
Cotopaxi                           2
Tungurahua                         2
Los Rios                           2
Loja                               1
Imbabura                           1
Chimborazo                         1
Santa Elena                        1
Esmeraldas                         1
Pastaza                            1
Bolivar                            1
Name: state, dtype: int64

In [37]:
pd.value_counts(df_stores.store_type)

D    18
C    15
A     9
B     8
E     4
Name: store_type, dtype: int64

In [38]:
pd.value_counts(df_stores.cluster)

3     7
6     6
10    6
15    5
14    4
13    4
1     3
11    3
8     3
4     3
2     2
9     2
7     2
5     1
16    1
12    1
17    1
Name: cluster, dtype: int64

In [39]:
df_stores.isnull().sum()

store_nbr     0
city          0
state         0
store_type    0
cluster       0
dtype: int64

In [40]:
df_train2 = pd.merge(df_train1, df_stores, how='left', on='store_nbr')

In [41]:
df_train2.shape

(125497040, 13)

In [42]:
df_train2.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
city           0
state          0
store_type     0
cluster        0
dtype: int64

In [43]:
df_train2.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1


In [44]:
df_oil = pd.read_csv('../data/oil.csv')

In [45]:
df_oil.shape

(1218, 2)

In [46]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [47]:
df_oil.isnull().sum()

date           0
dcoilwtico    43
dtype: int64

In [48]:
np.min(df_oil.dcoilwtico)

26.190000000000001

In [49]:
np.max(df_oil.dcoilwtico)

110.62

In [50]:
np.median(df_oil.dcoilwtico)

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


nan

In [51]:
df_oil_nan = df_oil[df_oil.dcoilwtico.isnull()]

In [52]:
df_oil_nan.shape

(43, 2)

In [53]:
df_oil_nan

,date,dcoilwtico
0,2013-01-01,NaN
14,2013-01-21,NaN
34,2013-02-18,NaN
63,2013-03-29,NaN
104,2013-05-27,NaN
132,2013-07-04,NaN
174,2013-09-02,NaN
237,2013-11-28,NaN
256,2013-12-25,NaN
261,2014-01-01,NaN


In [54]:
dfg = df_oil.groupby('dcoilwtico')

In [55]:
dfg.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [56]:
df_train2.dtypes

id               int64
date            object
store_nbr        int64
item_nbr         int64
unit_sales     float64
onpromotion     object
family          object
class            int64
perishable       int64
city            object
state           object
store_type      object
cluster          int64
dtype: object

In [57]:
df_oil.shape

(1218, 2)

In [58]:
np.datetime64(np.max(df_train2.date)) - np.datetime64(np.min(df_train2.date))

numpy.timedelta64(1687,'D')

In [59]:
np.datetime64(np.max(df_oil.date)) - np.datetime64(np.min(df_oil.date))

numpy.timedelta64(1703,'D')

In [60]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [61]:
df_oil.iloc[0]['dcoilwtico'] = 93.14

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [62]:
df_oil.head(10)

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [63]:
df_oil.loc[df_oil['date'] == '2013-01-01', 'dcoilwtico'] = 93.14

In [64]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [65]:
df_oil[df_oil['date'] == '2013-01-01']['dcoilwtico']

0    93.14
Name: dcoilwtico, dtype: float64

In [66]:
np.isnan(df_oil[df_oil['date'] == '2013-01-21']['dcoilwtico'])

14    True
Name: dcoilwtico, dtype: bool

In [67]:
print(df_oil[df_oil['date'] == '2017-06-30']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-01']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-02']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-03']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-04']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-05']['dcoilwtico'])

1173    46.02
Name: dcoilwtico, dtype: float64
Series([], Name: dcoilwtico, dtype: float64)
Series([], Name: dcoilwtico, dtype: float64)
1174   NaN
Name: dcoilwtico, dtype: float64
1175   NaN
Name: dcoilwtico, dtype: float64
1176    45.11
Name: dcoilwtico, dtype: float64


In [68]:
(46.02+45.11)/2.

45.565

In [69]:
df_oil.loc[df_oil['date'] == '2017-07-03', 'dcoilwtico'] = 45.565
df_oil.loc[df_oil['date'] == '2017-07-04', 'dcoilwtico'] = 45.565

In [70]:
print(df_oil[df_oil['date'] == '2017-06-30']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-01']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-02']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-03']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-04']['dcoilwtico'])
print(df_oil[df_oil['date'] == '2017-07-05']['dcoilwtico'])

1173    46.02
Name: dcoilwtico, dtype: float64
Series([], Name: dcoilwtico, dtype: float64)
Series([], Name: dcoilwtico, dtype: float64)
1174    45.565
Name: dcoilwtico, dtype: float64
1175    45.565
Name: dcoilwtico, dtype: float64
1176    45.11
Name: dcoilwtico, dtype: float64


In [71]:
df_oil.head()

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [72]:
pd.isnull(df_oil[df_oil['date'] == '2013-01-21']['dcoilwtico'])

14    True
Name: dcoilwtico, dtype: bool

In [73]:
pd.isnull(df_oil[df_oil['date'] == '2013-01-01']['dcoilwtico'])

0    False
Name: dcoilwtico, dtype: bool

In [74]:
pd.isnull(df_oil[df_oil['date'] == '2016-12-26']['dcoilwtico'])

1039    True
Name: dcoilwtico, dtype: bool

In [75]:
df_oil[df_oil['date'] == '2016-11-24']

,date,dcoilwtico
1017,2016-11-24,NaN


In [76]:
df_oil1 = df_oil.set_index('date')

In [77]:
df_oil1.head()

,dcoilwtico
date,
2013-01-01,93.14
2013-01-02,93.14
2013-01-03,92.97
2013-01-04,93.12
2013-01-07,93.20


In [78]:
df_oil1.loc['2013-01-01', 'dcoilwtico']

93.140000000000001

In [79]:
df_oil1.loc['2013-01-21', 'dcoilwtico']

nan

In [80]:
df_oil.iloc[0]['dcoilwtico']

93.140000000000001

In [81]:
df_oil.shape

(1218, 2)

In [82]:
np.isnan(df_oil1.loc['2013-01-21', 'dcoilwtico'])

True

In [83]:
# just set the oil price to prev ay price
# earlier thought of setting it to average of prev day and next day price
for i, date in enumerate(df_oil.date.values):
    next_day_price=0
    prev_day_price=0
    if i==0:
        continue
    prev_day_price = df_oil.iloc[i-1]['dcoilwtico']
    if(i < df_oil.shape[0]-1):
        next_day_price = df_oil.iloc[i-1]['dcoilwtico']
    else:
        continue
    
    curr_day_price = (prev_day_price + next_day_price)/2.0
    if np.isnan(df_oil1.loc[date, 'dcoilwtico']):
        df_oil.loc[df_oil['date'] == date, 'dcoilwtico'] = curr_day_price
    

In [84]:
df_oil.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

In [85]:
df_oil.head(50)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [86]:
df_oil.head(50)

,date,dcoilwtico
0,2013-01-01,93.14
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
5,2013-01-08,93.21
6,2013-01-09,93.08
7,2013-01-10,93.81
8,2013-01-11,93.60
9,2013-01-14,94.27


In [87]:
set(df_oil.date) - set(df_train2.date)

{'2013-12-25',
 '2014-12-25',
 '2015-12-25',
 '2017-08-16',
 '2017-08-17',
 '2017-08-18',
 '2017-08-21',
 '2017-08-22',
 '2017-08-23',
 '2017-08-24',
 '2017-08-25',
 '2017-08-28',
 '2017-08-29',
 '2017-08-30',
 '2017-08-31'}

In [88]:
l = list(set(df_oil.date) - set(df_train1.date))

In [89]:
df_oil.shape

(1218, 2)

In [90]:
df_oil2 = df_oil[~df_oil.date.isin(l)]

In [91]:
df_oil2.shape

(1203, 2)

In [92]:
len(np.unique(df_train1.date))

1684

In [93]:
len(set(df_train2.date))

1684

In [94]:
set(df_train2.date) - set(df_oil.date)

{'2013-01-05',
 '2013-01-06',
 '2013-01-12',
 '2013-01-13',
 '2013-01-19',
 '2013-01-20',
 '2013-01-26',
 '2013-01-27',
 '2013-02-02',
 '2013-02-03',
 '2013-02-09',
 '2013-02-10',
 '2013-02-16',
 '2013-02-17',
 '2013-02-23',
 '2013-02-24',
 '2013-03-02',
 '2013-03-03',
 '2013-03-09',
 '2013-03-10',
 '2013-03-16',
 '2013-03-17',
 '2013-03-23',
 '2013-03-24',
 '2013-03-30',
 '2013-03-31',
 '2013-04-06',
 '2013-04-07',
 '2013-04-13',
 '2013-04-14',
 '2013-04-20',
 '2013-04-21',
 '2013-04-27',
 '2013-04-28',
 '2013-05-04',
 '2013-05-05',
 '2013-05-11',
 '2013-05-12',
 '2013-05-18',
 '2013-05-19',
 '2013-05-25',
 '2013-05-26',
 '2013-06-01',
 '2013-06-02',
 '2013-06-08',
 '2013-06-09',
 '2013-06-15',
 '2013-06-16',
 '2013-06-22',
 '2013-06-23',
 '2013-06-29',
 '2013-06-30',
 '2013-07-06',
 '2013-07-07',
 '2013-07-13',
 '2013-07-14',
 '2013-07-20',
 '2013-07-21',
 '2013-07-27',
 '2013-07-28',
 '2013-08-03',
 '2013-08-04',
 '2013-08-10',
 '2013-08-11',
 '2013-08-17',
 '2013-08-18',
 '2013-08-

In [95]:
df = pd.DataFrame(list(set(df_train2.date) - set(df_oil.date)))

In [96]:
df.columns = ['date']

In [97]:
df['dcoilwtico'] = 0.0

In [98]:
df.head()

,date,dcoilwtico
0,2015-01-31,0.0
1,2015-03-07,0.0
2,2013-04-07,0.0
3,2017-06-25,0.0
4,2013-11-02,0.0


In [99]:
df.shape

(481, 2)

In [100]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

In [101]:
import datetime

In [102]:
df_oil2_dates = df_oil2.date.values

In [103]:
print(df_oil2_dates)

['2013-01-01' '2013-01-02' '2013-01-03' ..., '2017-08-11' '2017-08-14'
 '2017-08-15']


In [104]:
df_oil2.loc[df_oil2['date'] == '2015-01-30', 'dcoilwtico']

543    47.79
Name: dcoilwtico, dtype: float64

In [105]:
l = list(df_oil2_dates)

In [106]:
for i, date in enumerate(df.date.values):
    for j in range(1,5):
        new_date = str(np.datetime64(date) - np.timedelta64(j, 'D'))
        if new_date in l:
            print(new_date)
            last_price = df_oil2.loc[df_oil2['date'] == new_date, 'dcoilwtico']
            print(last_price.item())
            df.loc[df['date'] == date, 'dcoilwtico'] = last_price.item()
            break
        else:
            continue
        print('\n')

2015-01-30
47.79
2015-03-06
49.61
2013-04-05
92.76
2017-06-23
42.86
2013-11-01
94.56
2013-05-31
91.93
2016-08-19
48.48
2015-03-27
48.83
2013-07-12
105.85
2014-02-28
102.88
2017-07-14
46.53
2015-02-20
49.95
2014-09-26
95.55
2015-09-25
45.55
2014-11-14
75.91
2015-10-30
46.6
2013-06-14
97.83
2016-02-26
31.65
2014-06-20
107.95
2014-10-24
81.27
2014-05-23
105.01
2013-12-06
97.48
2016-03-04
35.91
2014-01-17
93.96
2015-11-27
40.57
2017-05-19
50.32
2015-05-01
59.1
2015-06-26
59.41
2013-09-13
108.31
2016-12-16
51.93
2015-12-18
34.72
2017-04-28
49.31
2014-07-11
101.48
2013-04-19
88.04
2015-06-05
59.11
2015-02-06
51.66
2017-07-07
44.25
2016-02-05
30.86
2016-02-12
29.32
2013-07-19
108.0
2013-12-13
96.27
2013-01-18
95.61
2015-07-03
56.93
2014-11-28
65.94
2013-10-25
97.4
2014-05-09
100.32
2015-08-07
43.87
2015-02-13
52.66
2015-06-19
59.62
2015-08-07
43.87
2015-05-15
59.73
2015-07-10
52.74
2016-03-11
38.51
2016-08-12
44.47
2013-09-20
104.7
2013-10-11
102.17
2013-03-01
90.71
2016-09-23
44.36
2013-07-2

In [107]:
df.head()

,date,dcoilwtico
0,2015-01-31,47.79
1,2015-03-07,49.61
2,2013-04-07,92.76
3,2017-06-25,42.86
4,2013-11-02,94.56


In [108]:
df.shape

(481, 2)

In [109]:
df_oil2.shape

(1203, 2)

In [110]:
len(set(df_train2.date))

1684

In [111]:
df_oil3 = pd.concat([df_oil2, df], axis=0)

In [112]:
df_oil3.isnull().sum()

date          0
dcoilwtico    0
dtype: int64

In [113]:
set(df_train2.date) - set(df_oil3.date)

set()

In [114]:
df_oil3.shape

(1684, 2)

In [115]:
df_train3 = pd.merge(df_train2, df_oil3, how='left', on='date')

In [116]:
df_train3.shape

(125497040, 14)

In [117]:
df_train3.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14


In [118]:
df_train3.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
city           0
state          0
store_type     0
cluster        0
dcoilwtico     0
dtype: int64

In [119]:
df_train3['onpromotion'] = df_train3['onpromotion'].astype('str')

In [120]:
# save the file as each step
feather.write_dataframe(df_train3, '../cache/train3.feather')

In [123]:
df_t = pd.read_csv('../data/transactions.csv')

In [124]:
df_t.head()

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


In [125]:
df_t.shape

(83488, 3)

In [126]:
len(set(df_t.store_nbr))

54

In [127]:
set(df_t.store_nbr) - set(df_train3.store_nbr)

set()

In [128]:
set(df_train3.store_nbr) - set(df_t.store_nbr)

set()

In [129]:
set(df_train3.date) - set(df_t.date)

{'2016-01-01', '2016-01-03'}

In [130]:
df_train3[df_train3['date'] == '2016-01-01'].shape

(1718, 14)

In [131]:
df_train3[df_train3['date'] == '2016-01-03'].shape

(100370, 14)

In [132]:
df_train4 = df_train3[df_train3['date'] != '2016-01-01' ]

In [133]:
df_train5 = df_train4[df_train4['date'] != '2016-01-03']

In [134]:
df_train5.shape

(125394952, 14)

In [135]:
df_t.isnull().sum()

date            0
store_nbr       0
transactions    0
dtype: int64

In [136]:
df_t[df_t['date'] == '2016-01-03']

,date,store_nbr,transactions


In [137]:
set(df_train5.store_nbr) - set(df_t.store_nbr)

set()

In [138]:
set(df_t.date) - set(df_train5.date)

set()

In [139]:
set(df_t.store_nbr) - set(df_train5.store_nbr)

set()

In [140]:
df_train5['ds'] = df_train5['date'].astype('str') + '_' + df_train5['store_nbr'].astype('str')

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [141]:
df_t['ds'] = df_t['date'].astype('str') + '_' + df_t['store_nbr'].astype('str')

In [142]:
df_t1 = df_t

In [143]:
df_t1 = df_t1.drop('date', axis=1)
df_t1 = df_t1.drop('store_nbr', axis=1)

In [144]:
df_t1.head()

,transactions,ds
0,770,2013-01-01_25
1,2111,2013-01-02_1
2,2358,2013-01-02_2
3,3487,2013-01-02_3
4,1922,2013-01-02_4


In [145]:
# (1236+1045+1055)/3

In [146]:
df_t1.shape

(83488, 2)

In [147]:
df_t1.loc[83487]

transactions              802
ds              2017-08-15_54
Name: 83487, dtype: object

In [148]:
# df_t1.loc[83488] = [1112, '2016-09-27_23']

In [149]:
# (663 + 690 + 776)/3
# df_t1.loc[83489] = [709, '2014-01-02_32']

In [150]:
df_t1.tail()

,transactions,ds
83483,2804,2017-08-15_50
83484,1573,2017-08-15_51
83485,2255,2017-08-15_52
83486,932,2017-08-15_53
83487,802,2017-08-15_54


In [151]:
df_t1.shape

(83488, 2)

In [152]:
df_train5.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,ds
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14,2013-01-01_25
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,2013-01-01_25
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,2013-01-01_25
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14,2013-01-01_25
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14,2013-01-01_25


In [153]:
df_train5.isnull().sum()

id             0
date           0
store_nbr      0
item_nbr       0
unit_sales     0
onpromotion    0
family         0
class          0
perishable     0
city           0
state          0
store_type     0
cluster        0
dcoilwtico     0
ds             0
dtype: int64

In [154]:
df_train6 = pd.merge(df_train5, df_t1, how='left', on='ds')

In [155]:
df_train6.isnull().sum()

id                   0
date                 0
store_nbr            0
item_nbr             0
unit_sales           0
onpromotion          0
family               0
class                0
perishable           0
city                 0
state                0
store_type           0
cluster              0
dcoilwtico           0
ds                   0
transactions    112537
dtype: int64

In [156]:
df_train6.dropna(inplace=True)

In [157]:
df_train6.isnull().sum()

id              0
date            0
store_nbr       0
item_nbr        0
unit_sales      0
onpromotion     0
family          0
class           0
perishable      0
city            0
state           0
store_type      0
cluster         0
dcoilwtico      0
ds              0
transactions    0
dtype: int64

In [158]:
df_train6.shape

(125282415, 16)

In [159]:
# feather.write_dataframe(df_train6, '../cache/train6.feather')

In [160]:
df_train6.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,ds,transactions
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14,2013-01-01_25,770.0
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,2013-01-01_25,770.0
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,2013-01-01_25,770.0
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14,2013-01-01_25,770.0
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14,2013-01-01_25,770.0


In [161]:
df_train6['transactions'] = df_train6['transactions'].astype('int')

In [162]:
df_train6 = df_train6.drop('ds', axis=1)

In [164]:
df_train6.shape

(125282415, 15)

In [163]:
# save the file as each step
feather.write_dataframe(df_train6, '../cache/train6.feather')

In [2]:
df_train6 = feather.read_dataframe('../cache/train6.feather')

In [3]:
df_train6.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,transactions
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14,770
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,770
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,770
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14,770
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14,770


In [4]:
df_train6.shape

(125282415, 15)

In [5]:
# df_train6.shape # org
#(125282415, 15)

In [6]:
df_h = pd.read_csv('../data/holidays_events.csv')

In [7]:
df_h.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [8]:
df_h.shape

(350, 6)

In [9]:
len(set(df_h.date))

312

In [10]:
len(set(df_train6.date))

1682

In [11]:
pd.value_counts(df_h.type)

Holiday       221
Event          56
Additional     51
Transfer       12
Work Day        5
Bridge          5
Name: type, dtype: int64

In [12]:
pd.value_counts(df_h.locale)

National    174
Local       152
Regional     24
Name: locale, dtype: int64

In [13]:
pd.value_counts(df_h.transferred)

False    338
True      12
Name: transferred, dtype: int64

In [14]:
# 2013-01-05 - Saturday
# 2013-01-12 - Saturday
# 2014-12-20 - Saturday
# 2015-01-10 - Saturday
# 2016-11-12 - Saturday
df_h[df_h.type == 'Work Day']

,date,type,locale,locale_name,description,transferred
42,2013-01-05,Work Day,National,Ecuador,Recupero puente Navidad,False
43,2013-01-12,Work Day,National,Ecuador,Recupero puente primer dia del ano,False
149,2014-12-20,Work Day,National,Ecuador,Recupero Puente Navidad,False
161,2015-01-10,Work Day,National,Ecuador,Recupero Puente Primer dia del ano,False
283,2016-11-12,Work Day,National,Ecuador,Recupero Puente Dia de Difuntos,False


In [15]:
df_h[(df_h.type == 'Holiday') & (df_h.transferred == True)]

,date,type,locale,locale_name,description,transferred
19,2012-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
135,2014-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True
255,2016-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
266,2016-07-25,Holiday,Local,Guayaquil,Fundacion de Guayaquil,True
268,2016-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True
297,2017-01-01,Holiday,National,Ecuador,Primer dia del ano,True
303,2017-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,True
312,2017-05-24,Holiday,National,Ecuador,Batalla de Pichincha,True
324,2017-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,True


In [16]:
df_h.loc[0]

date                   2012-03-02
type                      Holiday
locale                      Local
locale_name                 Manta
description    Fundacion de Manta
transferred                 False
Name: 0, dtype: object

In [17]:
df_h.type[(df_h.type == 'Holiday') & (df_h.transferred == True)] = 'Work Day'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_h[(df_h.type == 'Holiday') & (df_h.transferred == True)]

,date,type,locale,locale_name,description,transferred


In [19]:
# df_train6['type'] = 'Work Day'

In [20]:
# df_train6['description'] = 'Normal'

In [21]:
# df_train6 = df_train6.drop(['type', 'description'], axis=1)

In [22]:
df_h_national = df_h[df_h.locale == 'National']

In [23]:
df_h_national.shape

(174, 6)

In [24]:
df_h_national = df_h_national.drop_duplicates(['date'], keep='first')

In [25]:
df_h_national.shape

(168, 6)

In [26]:
pd.value_counts(df_h_national.type)

Event         53
Holiday       52
Additional    37
Work Day      13
Transfer       8
Bridge         5
Name: type, dtype: int64

In [27]:
df_h_national.type[(df_h_national.type != 'Event') & (df_h_national.type != 'Work Day')] = 'Holiday'

In [28]:
pd.value_counts(df_h_national.type)

Holiday     102
Event        53
Work Day     13
Name: type, dtype: int64

In [29]:
df_h_national.head()

,date,type,locale,locale_name,description,transferred
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
19,2012-10-09,Work Day,National,Ecuador,Independencia de Guayaquil,True
20,2012-10-12,Holiday,National,Ecuador,Traslado Independencia de Guayaquil,False
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


In [30]:
df_h_national = df_h_national.drop(['transferred', 'locale', 'description', 'locale_name'], axis=1)

In [31]:
df_h_national.head()

,date,type
14,2012-08-10,Holiday
19,2012-10-09,Work Day
20,2012-10-12,Holiday
21,2012-11-02,Holiday
22,2012-11-03,Holiday


In [32]:
df_train6['d1'] = df_train6['date'].values

In [33]:
df_train6["d1"] = (pd.to_datetime(df_train6["d1"]))

In [34]:
df_train6.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,transactions,d1
125282410,125497035,2017-08-15,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,Manabi,C,3,47.57,802,2017-08-15
125282411,125497036,2017-08-15,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,Manabi,C,3,47.57,802,2017-08-15
125282412,125497037,2017-08-15,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,Manabi,C,3,47.57,802,2017-08-15
125282413,125497038,2017-08-15,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,Manabi,C,3,47.57,802,2017-08-15
125282414,125497039,2017-08-15,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,Manabi,C,3,47.57,802,2017-08-15


In [35]:
df_train6["dom"] = df_train6["d1"].apply(lambda x: x.day)
df_train6["mon"] = df_train6["d1"].apply(lambda x: x.month)
df_train6["dow"] = df_train6["d1"].apply(lambda x: x.weekday())
df_train6["doy"] = df_train6["d1"].apply(lambda x: x.timetuple().tm_yday)
df_train6["dcount"] = df_train6["d1"].apply(lambda x: x.toordinal())

In [36]:
df_train6 = df_train6.drop('d1', axis=1)

In [37]:
df_train6.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,transactions,dom,mon,dow,doy,dcount
125282410,125497035,2017-08-15,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,Manabi,C,3,47.57,802,15,8,1,227,736556
125282411,125497036,2017-08-15,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,Manabi,C,3,47.57,802,15,8,1,227,736556
125282412,125497037,2017-08-15,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,Manabi,C,3,47.57,802,15,8,1,227,736556
125282413,125497038,2017-08-15,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,Manabi,C,3,47.57,802,15,8,1,227,736556
125282414,125497039,2017-08-15,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,Manabi,C,3,47.57,802,15,8,1,227,736556


In [38]:
np.min(df_train6['dow'])

0

In [39]:
np.max(df_train6['dow'])

6

In [40]:
df_h_national.head()

,date,type
14,2012-08-10,Holiday
19,2012-10-09,Work Day
20,2012-10-12,Holiday
21,2012-11-02,Holiday
22,2012-11-03,Holiday


In [41]:
# df_h_national["date"] = (pd.to_datetime(df_h_national["date"]))

In [42]:
df_train7 = pd.merge(df_train6, df_h_national, how='left', on='date')

In [43]:
df_train7.shape

(125282415, 21)

In [44]:
df_train6.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,state,store_type,cluster,dcoilwtico,transactions,dom,mon,dow,doy,dcount
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,Santa Elena,D,1,93.14,770,1,1,1,1,734869
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,770,1,1,1,1,734869
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,Santa Elena,D,1,93.14,770,1,1,1,1,734869
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,Santa Elena,D,1,93.14,770,1,1,1,1,734869
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,Santa Elena,D,1,93.14,770,1,1,1,1,734869


In [45]:
df_train6.shape

(125282415, 20)

In [46]:
df_train7.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,store_type,cluster,dcoilwtico,transactions,dom,mon,dow,doy,dcount,type
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,D,1,93.14,770,1,1,1,1,734869,Holiday
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,D,1,93.14,770,1,1,1,1,734869,Holiday
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,D,1,93.14,770,1,1,1,1,734869,Holiday
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,D,1,93.14,770,1,1,1,1,734869,Holiday
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,D,1,93.14,770,1,1,1,1,734869,Holiday


In [47]:
df_train7.isnull().sum()
# type            114199365

id                      0
date                    0
store_nbr               0
item_nbr                0
unit_sales              0
onpromotion             0
family                  0
class                   0
perishable              0
city                    0
state                   0
store_type              0
cluster                 0
dcoilwtico              0
transactions            0
dom                     0
mon                     0
dow                     0
doy                     0
dcount                  0
type            114199365
dtype: int64

In [48]:
df_train7[df_train7.type.isnull()].shape

(114199365, 21)

In [49]:
df_train7.type[(df_train7.type.isnull()) & (df_train7.dow == 5)] = 'Holiday'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
df_train7.type[(df_train7.type.isnull()) & (df_train7.dow == 6)] = 'Holiday'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_train7.isnull().sum()

id                     0
date                   0
store_nbr              0
item_nbr               0
unit_sales             0
onpromotion            0
family                 0
class                  0
perishable             0
city                   0
state                  0
store_type             0
cluster                0
dcoilwtico             0
transactions           0
dom                    0
mon                    0
dow                    0
doy                    0
dcount                 0
type            79822467
dtype: int64

In [52]:
df_train7.fillna('Work Day', inplace=True)

In [53]:
df_train7.isnull().sum()

id              0
date            0
store_nbr       0
item_nbr        0
unit_sales      0
onpromotion     0
family          0
class           0
perishable      0
city            0
state           0
store_type      0
cluster         0
dcoilwtico      0
transactions    0
dom             0
mon             0
dow             0
doy             0
dcount          0
type            0
dtype: int64

In [54]:
df_h_national = df_h[df_h.locale == 'National']

In [55]:
df_h_national.shape

(174, 6)

In [56]:
df_h_national.type[(df_h_national.type != 'Event') & (df_h_national.type != 'Work Day')] = 'Holiday'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/pandas/core/generic.py:5088: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2910: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exe

In [57]:
df_h_national.head()

,date,type,locale,locale_name,description,transferred
14,2012-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False
19,2012-10-09,Work Day,National,Ecuador,Independencia de Guayaquil,True
20,2012-10-12,Holiday,National,Ecuador,Traslado Independencia de Guayaquil,False
21,2012-11-02,Holiday,National,Ecuador,Dia de Difuntos,False
22,2012-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False


In [58]:
df_h_national = df_h_national.drop(['transferred', 'locale', 'type', 'locale_name'], axis=1)

In [59]:
df_h_national.head()

,date,description
14,2012-08-10,Primer Grito de Independencia
19,2012-10-09,Independencia de Guayaquil
20,2012-10-12,Traslado Independencia de Guayaquil
21,2012-11-02,Dia de Difuntos
22,2012-11-03,Independencia de Cuenca


In [60]:
df_train8 = pd.merge(df_train7, df_h_national, how='left', on='date')

In [61]:
df_train8.isnull().sum()

id                      0
date                    0
store_nbr               0
item_nbr                0
unit_sales              0
onpromotion             0
family                  0
class                   0
perishable              0
city                    0
state                   0
store_type              0
cluster                 0
dcoilwtico              0
transactions            0
dom                     0
mon                     0
dow                     0
doy                     0
dcount                  0
type                    0
description     114199365
dtype: int64

In [62]:
df_train8.type[(df_train8.description.isnull()) & (df_train8.dow == 5)] = 'Weekend'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
df_train8.type[(df_train8.description.isnull()) & (df_train8.dow == 6)] = 'Weekend'

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [64]:
df_train8.fillna('Normal', inplace=True)

In [65]:
df_train8.isnull().sum()

id              0
date            0
store_nbr       0
item_nbr        0
unit_sales      0
onpromotion     0
family          0
class           0
perishable      0
city            0
state           0
store_type      0
cluster         0
dcoilwtico      0
transactions    0
dom             0
mon             0
dow             0
doy             0
dcount          0
type            0
description     0
dtype: int64

In [66]:
df_train8.shape

(125665673, 22)

In [67]:
df_train8.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,cluster,dcoilwtico,transactions,dom,mon,dow,doy,dcount,type,description
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,93.14,770,1,1,1,1,734869,Holiday,Primer dia del ano
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,93.14,770,1,1,1,1,734869,Holiday,Primer dia del ano
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,93.14,770,1,1,1,1,734869,Holiday,Primer dia del ano
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,93.14,770,1,1,1,1,734869,Holiday,Primer dia del ano
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,93.14,770,1,1,1,1,734869,Holiday,Primer dia del ano


In [68]:
feather.write_dataframe(df_train8, '../cache/train8.feather')

In [2]:
df_train8 = feather.read_dataframe('../cache/train8.feather')

In [4]:
df_h = pd.read_csv('../data/holidays_events.csv')

In [5]:
df_h_local = df_h[df_h.locale == 'Local']

In [6]:
df_h_local.shape

(152, 6)

In [7]:
len(set(df_h_local.locale_name))

19

In [8]:
len(set(df_train8.city))

22

In [9]:
set(df_train8.city) - set(df_h_local.locale_name)

{'Babahoyo', 'Daule', 'Playas'}

In [10]:
df_h_local = df_h_local.drop_duplicates(['date'], keep='first')

In [11]:
df_h_local.shape

(138, 6)

In [12]:
df_h_local.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
5,2012-05-12,Holiday,Local,Puyo,Cantonizacion del Puyo,False


In [13]:
df_h_l1 = df_h_local.set_index('date')

In [14]:
import os
x = {i for i in range(20)}
os.sched_setaffinity(0, x)

In [15]:
def set_train_type_desc_local(date):
    t = df_h_l1.loc[date].type
    d = df_h_l1.loc[date].description
    c = df_h_l1.loc[date].locale_name

    df_train8.type[(df_train8.date == date) & (df_train8.city == c)] = t
    df_train8.description[(df_train8.date == date) & (df_train8.city == c)] = d

In [16]:
for date in tqdm(df_h_local.date):
    set_train_type_desc_local(date) 

  0%|          | 0/138 [00:00<?, ?it/s]/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
100%|██████████| 138/138 [2:03:47<00:00, 53.82s/it] 


In [17]:
df_h_regional = df_h[df_h.locale == 'Regional']

In [18]:
df_h_regional.shape

(24, 6)

In [19]:
len(set(df_h_regional.locale_name))

4

In [20]:
len(set(df_train8.state))

16

In [21]:
set(df_train8.state) - set(df_h_regional.locale_name)

{'Azuay',
 'Bolivar',
 'Chimborazo',
 'El Oro',
 'Esmeraldas',
 'Guayas',
 'Loja',
 'Los Rios',
 'Manabi',
 'Pastaza',
 'Pichincha',
 'Tungurahua'}

In [22]:
len(set(df_train8.state) - set(df_h_regional.locale_name))

12

In [23]:
df_h_regional = df_h_regional.drop_duplicates(['date'], keep='first')

In [24]:
df_h_regional.shape

(24, 6)

In [25]:
df_h_regional.head()

,date,type,locale,locale_name,description,transferred
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
7,2012-06-25,Holiday,Regional,Imbabura,Provincializacion de Imbabura,False
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas,Provincializacion de Santo Domingo,False
24,2012-11-07,Holiday,Regional,Santa Elena,Provincializacion Santa Elena,False
47,2013-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False


In [26]:
df_h_r1 = df_h_regional.set_index('date')

In [27]:
def set_train_type_desc_regional(date):
    t = df_h_r1.loc[date].type
    d = df_h_r1.loc[date].description
    s = df_h_r1.loc[date].locale_name

    df_train8.type[(df_train8.date == date) & (df_train8.state == s)] = t
    df_train8.description[(df_train8.date == date) & (df_train8.state == s)] = d

In [28]:
for date in tqdm(df_h_regional.date):
    set_train_type_desc_regional(date) 

  0%|          | 0/24 [00:00<?, ?it/s]/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
100%|██████████| 24/24 [21:32<00:00, 53.85s/it]


In [29]:
df_train8.tail()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,cluster,dcoilwtico,transactions,dom,mon,dow,doy,dcount,type,description
125665668,125497035,2017-08-15,54,2089339,4.0,False,GROCERY I,1006,0,El Carmen,...,3,47.57,802,15,8,1,227,736556,Work Day,Normal
125665669,125497036,2017-08-15,54,2106464,1.0,True,BEVERAGES,1148,0,El Carmen,...,3,47.57,802,15,8,1,227,736556,Work Day,Normal
125665670,125497037,2017-08-15,54,2110456,192.0,False,BEVERAGES,1120,0,El Carmen,...,3,47.57,802,15,8,1,227,736556,Work Day,Normal
125665671,125497038,2017-08-15,54,2113914,198.0,True,CLEANING,3040,0,El Carmen,...,3,47.57,802,15,8,1,227,736556,Work Day,Normal
125665672,125497039,2017-08-15,54,2116416,2.0,False,GROCERY I,1060,0,El Carmen,...,3,47.57,802,15,8,1,227,736556,Work Day,Normal


In [30]:
df_train8.shape

(125665673, 22)

In [31]:
df_train9 = df_train8

In [32]:
df_train9 = df_train9.reset_index(drop=True)

In [33]:
df_train9.shape

(125665673, 22)

In [34]:
feather.write_dataframe(df_train9, '../cache/train9.feather')

In [35]:
df_train9['pd'] = 0 # payday

In [36]:
df_train9.pd[(df_train9.dom == 15)]= 1

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 1)]= 1
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 3)]= 1
df_train9.pd[(df_train9.dom == 30) & (df_train9.mon == 4)]= 1
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 5)]= 1
df_train9.pd[(df_train9.dom == 30) & (df_train9.mon == 6)]= 1
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 7)]= 1
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 8)]= 1
df_train9.pd[(df_train9.dom == 30) & (df_train9.mon == 9)]= 1
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 10)]= 1
df_train9.pd[(df_train9.dom == 30) & (df_train9.mon == 11)]= 1
df_train9.pd[(df_train9.dom == 31) & (df_train9.mon == 12)]= 1

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we c

In [38]:
df_train9.pd[df_train9.date == '2013-02-28']= 1
df_train9.pd[df_train9.date == '2014-02-28']= 1
df_train9.pd[df_train9.date == '2015-02-28']= 1
df_train9.pd[df_train9.date == '2016-02-29']= 1
df_train9.pd[df_train9.date == '2017-02-28']= 1

/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/watts/anaconda3/envs/rss/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we c

In [39]:
df_train9.shape

(125665673, 23)

In [40]:
df_train9.isnull().sum()

id              0
date            0
store_nbr       0
item_nbr        0
unit_sales      0
onpromotion     0
family          0
class           0
perishable      0
city            0
state           0
store_type      0
cluster         0
dcoilwtico      0
transactions    0
dom             0
mon             0
dow             0
doy             0
dcount          0
type            0
description     0
pd              0
dtype: int64

In [41]:
np.datetime64('2016-04-16') - np.datetime64('2013-01-01')

numpy.timedelta64(1201,'D')

In [45]:
(np.datetime64('2016-04-16') - np.datetime64('2016-04-08'))/np.timedelta64(7, 'D')

1.1428571428571428

In [47]:
int(np.ceil((np.datetime64('2016-04-16') - np.datetime64('2016-04-08'))/np.timedelta64(7, 'D')))

2

In [50]:
def weeks_before_earthquake(date):
    if np.datetime64('2016-04-16') <= np.datetime64(date):
        return 0
    return int(np.ceil((np.datetime64('2016-04-16') - np.datetime64(date))/np.timedelta64(7, 'D')))

In [51]:
def weeks_after_earthquake(date):
    if np.datetime64('2016-04-16') >= np.datetime64(date):
        return 0
    return int(np.ceil((np.datetime64(date) - np.datetime64('2016-04-16'))/np.timedelta64(7, 'D')))

In [52]:
df_train9['wbe'] = 0
df_train9['wae'] = 0

In [53]:
df_train9['wbe'] = df_train9['date'].map(lambda d: weeks_before_earthquake(d))

In [54]:
df_train9['wae'] = df_train9['date'].map(lambda d: weeks_after_earthquake(d))

In [55]:
df_train9.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,1,1,734869,Holiday,Primer dia del ano,0,172,0


In [56]:
df_train9[df_train9.date == '2016-04-16'].shape

(100966, 25)

In [57]:
df_train9[df_train9.date == '2016-04-16'].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
76275975,76410846,2016-04-16,1,103665,4.0,False,BREAD/BAKERY,2712,1,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76275976,76410847,2016-04-16,1,105574,11.0,False,GROCERY I,1045,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76275977,76410848,2016-04-16,1,105575,7.0,False,GROCERY I,1045,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76275978,76410849,2016-04-16,1,105577,3.0,False,GROCERY I,1045,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0
76275979,76410850,2016-04-16,1,105737,3.0,False,GROCERY I,1044,0,Quito,...,16,4,5,107,736070,Event,Terremoto Manabi,0,0,0


In [58]:
df_train9[df_train9.date == '2016-04-17'].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
76376941,76511812,2016-04-17,1,105574,32.0,False,GROCERY I,1045,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76376942,76511813,2016-04-17,1,105575,6.0,False,GROCERY I,1045,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76376943,76511814,2016-04-17,1,105577,12.0,False,GROCERY I,1045,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76376944,76511815,2016-04-17,1,105737,6.0,False,GROCERY I,1044,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1
76376945,76511816,2016-04-17,1,105857,1.0,False,GROCERY I,1092,0,Quito,...,17,4,6,108,736071,Event,Terremoto Manabi+1,0,0,1


In [59]:
df_train9[df_train9.date == '2016-04-15'].head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,dom,mon,dow,doy,dcount,type,description,pd,wbe,wae
76182035,76316906,2016-04-15,1,103665,4.0,False,BREAD/BAKERY,2712,1,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76182036,76316907,2016-04-15,1,105574,3.0,False,GROCERY I,1045,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76182037,76316908,2016-04-15,1,105575,13.0,False,GROCERY I,1045,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76182038,76316909,2016-04-15,1,105577,2.0,False,GROCERY I,1045,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0
76182039,76316910,2016-04-15,1,105857,13.0,False,GROCERY I,1092,0,Quito,...,15,4,4,106,736069,Work Day,Normal,1,1,0


In [60]:
df_train9['wfe'] = df_train9['wbe'].astype(int) + df_train9['wae'].astype(int)

In [61]:
df_train9.head()

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family,class,perishable,city,...,mon,dow,doy,dcount,type,description,pd,wbe,wae,wfe
0,0,2013-01-01,25,103665,7.0,Unk,BREAD/BAKERY,2712,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
1,1,2013-01-01,25,105574,1.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
2,2,2013-01-01,25,105575,2.0,Unk,GROCERY I,1045,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
3,3,2013-01-01,25,108079,1.0,Unk,GROCERY I,1030,0,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172
4,4,2013-01-01,25,108701,1.0,Unk,DELI,2644,1,Salinas,...,1,1,1,734869,Holiday,Primer dia del ano,0,172,0,172


In [62]:
feather.write_dataframe(df_train9, '../cache/train10.feather')

In [3]:
df_train9 = feather.read_dataframe('../cache/train10.feather')

In [4]:
df_train9.shape

(125665673, 26)

In [5]:
df_train10 = df_train9.drop_duplicates(['id'], keep='first')

In [6]:
df_train10.shape

(125282415, 26)

In [7]:
feather.write_dataframe(df_train10, '../cache/train10.feather')

In [8]:
# create a sample data frame

In [9]:
df_10 = df_train10.sample(frac=0.1, replace=False)

In [10]:
df_10.shape

(12528242, 26)

In [11]:
feather.write_dataframe(df_10, '../cache/train10_sample_10.feather')

In [12]:
df_1 = df_train10.sample(frac=0.01, replace=False)

In [13]:
df_1.shape

(1252824, 26)

In [14]:
feather.write_dataframe(df_1, '../cache/train10_sample_1.feather')